## import

In [ ]:
import os
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
import re
import pandas as pd

##Preprocess(1)

In [ ]:
def preprocess_images(input_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    for filename in os.listdir(input_dir):
        input_path = os.path.join(input_dir, filename)

        if not filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            print(f"Skipping non-image file: {filename}")
            continue

        print(f"Processing {filename}...")
        image = cv2.imread(input_path)

        if image is None:
            print(f"Failed to load image: {input_path}. Skipping...")
            continue

        green_channel = image[:, :, 1]
        dark_threshold = 30
        brightened_img = np.where(
            (green_channel > 0) & (green_channel < dark_threshold),
            green_channel + (dark_threshold - green_channel),
            green_channel
        )
        brightened_img = np.clip(brightened_img, 0, 255).astype(np.uint8)

        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        enhanced_img = clahe.apply(brightened_img)

        blurred_img = cv2.GaussianBlur(enhanced_img, (5, 5), sigmaX=1.0)

        kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
        sharpened_img = cv2.filter2D(blurred_img, -1, kernel)

        output_path = os.path.join(output_dir, filename)
        cv2.imwrite(output_path, sharpened_img)
        print(f"Saved preprocessed image to {output_path}")

def main():
    input_dir = input("Enter the input directory path: ").strip()
    output_dir = input("Enter the output directory path: ").strip()

    if not os.path.exists(input_dir):
        print("The input directory does not exist.")
        return

    preprocess_images(input_dir, output_dir)

if __name__ == "__main__":
    main()

##Preprocess(2)

In [ ]:
def preprocess_images(input_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)

    for filename in os.listdir(input_dir):
        input_path = os.path.join(input_dir, filename)

        if not filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            print(f"Skipping non-image file: {filename}")
            continue

        print(f"Processing {filename}...")
        image = cv2.imread(input_path)

        if image is None:
            print(f"Failed to load image: {input_path}. Skipping...")
            continue

        green_channel = image[:, :, 1]

        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        enhanced_img = clahe.apply(green_channel)

        blurred_img = cv2.GaussianBlur(enhanced_img, (5, 5), sigmaX=1.0)

        kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
        sharpened_img = cv2.filter2D(blurred_img, -1, kernel)

        output_path = os.path.join(output_dir, filename)
        cv2.imwrite(output_path, sharpened_img)
        print(f"Saved preprocessed image to {output_path}")

input_dir = "/content/retina_image"
output_dir = "preprocess2/"
preprocess_images(input_dir, output_dir)

##Preprocess(3)

In [ ]:
def preprocess_images(input_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)

    for filename in os.listdir(input_dir):
        input_path = os.path.join(input_dir, filename)

        if not filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            print(f"Skipping non-image file: {filename}")
            continue

        print(f"Processing {filename}...")
        image = cv2.imread(input_path)

        if image is None:
            print(f"Failed to load image: {input_path}. Skipping...")
            continue

        green_channel = image[:, :, 1]

        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        enhanced_img = clahe.apply(green_channel)

        blurred_img = cv2.GaussianBlur(enhanced_img, (5, 5), sigmaX=1.0)

        kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
        sharpened_img = cv2.filter2D(blurred_img, -1, kernel)

        brightness_boost = 50
        adjusted_img = sharpened_img.copy()
        mask = (sharpened_img > 0)
        adjusted_img[mask] = np.clip(sharpened_img[mask] + brightness_boost, 0, 255)

        output_path = os.path.join(output_dir, filename)
        cv2.imwrite(output_path, adjusted_img)
        print(f"Saved preprocessed image to {output_path}")

input_dir = "/content/retina_image/"  # 입력 이미지 디렉토리
output_dir = "preprocess3/"  # 출력 디렉토리
preprocess_images(input_dir, output_dir)


##Run TEED

In [ ]:
!python TEED/main.py

##Postprocess(1) - Median

In [ ]:
def apply_median_and_crop_with_background(input_dir, output_dir, radius_scale=2.2):
    os.makedirs(output_dir, exist_ok=True)  # 출력 디렉토리 생성

    for filename in os.listdir(input_dir):
        input_path = os.path.join(input_dir, filename)
        output_path = os.path.join(output_dir, filename)

        if not filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            print(f"Skipping non-image file: {filename}")
            continue

        print(f"Processing {filename}...")

        image = cv2.imread(input_path)
        if image is None:
            print(f"Failed to load image: {input_path}. Skipping...")
            continue

        median_filtered = cv2.medianBlur(image, 5)

        border_pixels = np.concatenate([
            image[0, :, :],
            image[-1, :, :],
            image[:, 0, :],
            image[:, -1, :]
        ])
        background_color = border_pixels.mean(axis=0).astype(np.uint8)

        height, width = image.shape[:2]
        center = (width // 2, height // 2)
        radius = int(min(width, height) / radius_scale)

        mask = np.zeros((height, width), dtype=np.uint8)
        cv2.circle(mask, center, radius, 255, -1)

        circular_cropped = cv2.bitwise_and(median_filtered, median_filtered, mask=mask)

        result = np.zeros_like(image)
        for i in range(3):
            result[:, :, i] = np.where(mask == 255, circular_cropped[:, :, i], background_color[i])

        cv2.imwrite(output_path, result)
        print(f"Saved processed and cropped image to {output_path}")

input_dir = "/content/zip/result/TEED/preprocess3"
output_dir = "/content/zip/result/median/preprocess3"
apply_median_and_crop_with_background(input_dir, output_dir)

##Postprocess(2) - Median, Bilateral

In [ ]:
def apply_filters_and_crop(input_dir, output_dir, background_color=(255, 255, 255), radius_scale=2.2):
    os.makedirs(output_dir, exist_ok=True)

    for filename in os.listdir(input_dir):
        input_path = os.path.join(input_dir, filename)
        output_path = os.path.join(output_dir, filename)

        if not filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            print(f"Skipping non-image file: {filename}")
            continue

        print(f"Processing {filename}...")

        image = cv2.imread(input_path)
        if image is None:
            print(f"Failed to load image: {input_path}. Skipping...")
            continue

        grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        median_filtered = cv2.medianBlur(grayscale_image, 5)
        bilateral_filtered = cv2.bilateralFilter(median_filtered, d=9, sigmaColor=75, sigmaSpace=75)

        image[:, :, 0] = bilateral_filtered
        image[:, :, 1] = bilateral_filtered
        image[:, :, 2] = bilateral_filtered

        height, width = image.shape[:2]
        result = np.full_like(image, background_color)
        center = (width // 2, height // 2)
        radius = min(width, height) / radius_scale

        mask = np.zeros((height, width), dtype=np.uint8)
        cv2.circle(mask, center, int(radius), 255, -1)

        circular_cropped = cv2.bitwise_and(image, image, mask=mask)
        for i in range(3):  # Assuming the image has 3 color channels
            result[:, :, i] = np.where(mask == 255, circular_cropped[:, :, i], result[:, :, i])

        # 결과 저장
        cv2.imwrite(output_path, result)
        print(f"Saved processed and cropped image to {output_path}")

# Example usage
input_dir = "/content/result/TEED/preprocess3"  # 입력 이미지 디렉토리
output_dir = "/content/result/final/postprocess3"  # 출력 디렉토리
apply_filters_and_crop(input_dir, output_dir)

##Postprocess(2-1) - Add sharpening

In [ ]:
def sharpen_images(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    sharpening_kernel = np.array([[-1, -1, -1],
                                   [-1, 9, -1],
                                   [-1, -1, -1]])

    for filename in os.listdir(input_folder):
        input_path = os.path.join(input_folder, filename)

        if not filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            print(f"Skipping non-image file: {filename}")
            continue

        print(f"Processing {filename}...")

        image = cv2.imread(input_path)
        if image is None:
            print(f"Failed to load image: {input_path}")
            continue

        sharpened = cv2.filter2D(image, -1, sharpening_kernel)

        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, sharpened)
        print(f"Saved sharpened image to {output_path}")

input_folder = "/content/result/final/postprocess3"
output_folder = "/content/result/final_sharpening/postprocess3"
sharpen_images(input_folder, output_folder)

##Evaluation

In [ ]:
def compare_images(image1_path, image2_path):
    image1 = cv2.imread(image1_path, cv2.IMREAD_GRAYSCALE)
    image2 = cv2.imread(image2_path, cv2.IMREAD_GRAYSCALE)

    if image1 is None or image2 is None:
        raise ValueError(f"One or both images could not be loaded: {image1_path}, {image2_path}")
    if image1.shape != image2.shape:
        image2 = cv2.resize(image2, (image1.shape[1], image1.shape[0]))

    # SSIM
    ssim_score, _ = ssim(image1, image2, full=True)

    # MSE
    mse_score = np.mean((image1 - image2) ** 2)

    # NCC
    ncc_score = np.corrcoef(image1.flatten(), image2.flatten())[0, 1]

    # Pixel Overlap
    overlap_score = np.sum(image1 == image2) / image1.size

    return ssim_score, mse_score, ncc_score, overlap_score

def extract_prefix(filename):
    """
    Extract the prefix (e.g., Image_01L) from a filename.
    """
    match = re.match(r"(Image_\d+[LR])", filename)
    return match.group(1) if match else None

def compare_folders_by_prefix(folder1, folder2, output_csv_path="comparison_results.csv"):
    if not os.path.exists(folder1) or not os.path.exists(folder2):
        raise ValueError("One or both folders do not exist.")

    folder1_files = sorted([f for f in os.listdir(folder1) if f.endswith(('.png', '.jpg', '.jpeg'))])
    folder2_files = sorted([f for f in os.listdir(folder2) if f.endswith(('.png', '.jpg', '.jpeg'))])

    folder1_map = {extract_prefix(f): f for f in folder1_files if extract_prefix(f)}
    folder2_map = {extract_prefix(f): f for f in folder2_files if extract_prefix(f)}

    results = []

    for prefix, file1 in folder1_map.items():
        if prefix not in folder2_map:
            print(f"No match found for {prefix} in folder2.")
            continue

        path1 = os.path.join(folder1, file1)
        path2 = os.path.join(folder2, folder2_map[prefix])

        try:
            ssim_score, mse_score, ncc_score, overlap_score = compare_images(path1, path2)
            results.append({
                "Prefix": prefix,
                "Folder1 File": file1,
                "Folder2 File": folder2_map[prefix],
                "SSIM": ssim_score,
                "MSE": mse_score,
                "NCC": ncc_score,
                "Pixel Overlap": overlap_score
            })

        except ValueError as e:
            print(f"Error comparing {file1} with {folder2_map[prefix]}: {e}")

    results_df = pd.DataFrame(results)
    if not results_df.empty:
        print("\n=== Comparison Results ===")
        print(results_df)

        avg_ssim = results_df["SSIM"].mean()
        avg_mse = results_df["MSE"].mean()
        avg_ncc = results_df["NCC"].mean()
        avg_overlap = results_df["Pixel Overlap"].mean()

        overall_averages = {
            "Prefix": "Overall Average",
            "Folder1 File": "-",
            "Folder2 File": "-",
            "SSIM": avg_ssim,
            "MSE": avg_mse,
            "NCC": avg_ncc,
            "Pixel Overlap": avg_overlap
        }
        results_df = pd.concat([results_df, pd.DataFrame([overall_averages])], ignore_index=True)

        # Save results to a CSV file
        results_df.to_csv(output_csv_path, index=False)
        print(f"\nResults saved to {output_csv_path}")

        print("\n=== Overall Averages ===")
        print(f"Average SSIM: {avg_ssim:.4f}")
        print(f"Average MSE: {avg_mse:.4f}")
        print(f"Average NCC: {avg_ncc:.4f}")
        print(f"Average Pixel Overlap: {avg_overlap:.4f}")
    else:
        print("No valid comparisons found. No CSV file was created.")

folder1 = "/content/zip/result/median/preprocess3"
folder2 = "/content/zip/result/masking"
output_csv_path = "/content/median3_results.csv"
compare_folders_by_prefix(folder1, folder2, output_csv_path)
